In [2]:
import numpy as np
import matplotlib as plt

In [3]:
# 计算函数 f 在 (a,b) 上的 Romberg 积分，外推 n-1 次
def RombergIntegration(f,a,b,n):
    
    #存储各次外推数值
    repository = np.zeros((n,n),dtype = np.float64)
    
    #初始化
    h = b - a
    repository[0,0] = (f(a) + f(b)) * h / 2
    
    #开始计算
    for i in range(1,n):
        h = h / 2
        # 步长减半
        for j in range(i+1):
            if j == 0:
            #第 0 列是梯形积分的递推
                repository[i,j] = 1/2 * repository[i-1,j] + h * InnerSum(f,a,b,h)
                continue
            #依次线性组合
            repository[i,j] = (4**(j) * repository[i,j-1] - 
                               repository[i-1,j-1]) / (4**(j) - 1)
    
    return repository

#梯形积分递推辅助函数
def InnerSum(f,a,b,h):
    #计算函数的“内点和"
    number = (b-a)/h + 1
    InervalDivide = np.linspace(a,b,int(number))
    PointsShouldCalculate = InervalDivide[1::2]
    return np.sum(f(PointsShouldCalculate)) 

In [12]:
f = lambda x: 1/x  
RombergIntegration(f,1,2,5)

array([[0.75      , 0.        , 0.        , 0.        , 0.        ],
       [0.70833333, 0.69444444, 0.        , 0.        , 0.        ],
       [0.69702381, 0.69325397, 0.6931746 , 0.        , 0.        ],
       [0.69412185, 0.69315453, 0.6931479 , 0.69314748, 0.        ],
       [0.6933912 , 0.69314765, 0.69314719, 0.69314718, 0.69314718]])

In [4]:
# 复化梯形积分
def TrapezoidalIntegration(f,a,b,l):
    h = (b - a) / 2**l
    points = np.linspace(a,b,2**l+1)
    
    innerpoints = points[1:-1]
    innerSum = np.sum(f(innerpoints))
    result = h / 2 * (f(a) + f(b) + 2 * innerSum)
    
    return result

def SimpsonIntegration(f,a,b,l):
    
    h = (b - a) / 2**(l+1)
    points = np.linspace(a,b,2**(l+1)+1) #分 2**(l)+1 段，内点有 2**(l+1)+1 个
    
    innerOddpoints = points[1:-1:2]
    innerEvenpoints = points[2:-1:2]
    
    oddSum = np.sum(f(innerOddpoints))
    evenSum = np.sum(f(innerEvenpoints))
    
    result = h / 3 * (f(a) + f(b) + 4 * oddSum + 2 * evenSum)
    return result

In [13]:
f = lambda x : np.sin(x)
accurate = 0.256640120404913452934297435929 
# Mathematica 输出的30位小数，精度不够会导致后面 SimpsonOrder 出现 inf

TrapezoidalResult = np.array([TrapezoidalIntegration(f,1,5,i) for i in range(1,13)])
SimpsonResult = np.array([SimpsonIntegration(f,1,5,i) for i in range(1,13)])

TrapezoidalError = TrapezoidalResult - accurate
SimpsonError = SimpsonResult - accurate

In [8]:
TrapezoidalOrder = np.array([np.log(TrapezoidalError[i] / TrapezoidalError[i+1]) / np.log(2) for i in range(11)])

SimpsonOrder = np.array([np.log(SimpsonError[i] / SimpsonError[i+1]) / np.log(2) for i in range(11)])

[2.07819653 2.01838958 2.00453033 2.00112847 2.00028186 2.00007045
 2.00001761 2.0000044  2.0000011  2.00000028 2.00000007]
[4.13667562 4.03266862 4.00807948 4.00201449 4.00050329 4.00012527
 4.0000179  4.00006031 4.00677096 3.95419631        inf]


/var/folders/_r/s4ymnyp94kg4jf8pzqddcwmh0000gn/T/ipykernel_14120/1320071230.py:5: RuntimeWarning: divide by zero encountered in double_scalars
  SimpsonOrder = np.array([np.log(SimpsonError[i] / SimpsonError[i+1]) / np.log(2) for i in range(11)])


In [10]:
for i in range(12):
    if i == 0:
        print("Trapezoidal\n")
        print("result = ",TrapezoidalResult[i],"\terror  =",TrapezoidalError[i])
        continue
    print("result = ",TrapezoidalResult[i],"\terror = ",TrapezoidalError[i],"\torder = ",TrapezoidalOrder[i-1])

Trapezoidal

result =  0.16478672626449248 	error  = -0.09185339414042099
result =  0.23488829464999966 	error =  -0.0217518257549138 	order =  2.0781965327026297
result =  0.2512710400016469 	error =  -0.005369080403266557 	order =  2.0183895847975637
result =  0.25530205867416406 	error =  -0.001338061730749407 	order =  2.0045303342994902
result =  0.25630586652647214 	error =  -0.00033425387844132626 	order =  2.0011284699420995
result =  0.25655657325963355 	error =  -8.354714527991636e-05 	order =  2.0002818615756546
result =  0.25661923463850833 	error =  -2.0885766405132067e-05 	order =  2.0000704494217487
result =  0.25663489902705144 	error =  -5.22137786201915e-06 	order =  2.00001761139704
result =  0.2566388150644316 	error =  -1.3053404818719194e-06 	order =  2.0000044028042443
result =  0.256639794070042 	error =  -3.263348714588332e-07 	order =  2.000001100845038
result =  0.2566400388212112 	error =  -8.158370223831923e-08 	order =  2.000000276331072
result =  0.256640

In [11]:
for i in range(12):
    if i == 0:
        print("Simpson\n")
        print("result = ",SimpsonResult[i],"\terror  =",SimpsonError[i])
        continue
    print("result = ",SimpsonResult[i],"\terror = ",SimpsonError[i],"\torder = ",SimpsonOrder[i-1])

Simpson

result =  0.25825548411183546 	error  = 0.0016153637069220017
result =  0.2567319551188626 	error =  9.18347139491349e-05 	order =  4.1366756188335385
result =  0.25664573156500303 	error =  5.611160089569189e-06 	order =  4.0326686227613875
result =  0.2566404691439082 	error =  3.487389947376407e-07 	order =  4.00807947909291
result =  0.2566401421706871 	error =  2.1765773627890894e-08 	order =  4.00201448577115
result =  0.25664012176479983 	error =  1.3598863701780317e-09 	order =  4.000503286761924
result =  0.256640120489899 	error =  8.498551862246018e-11 	order =  4.000125267524913
result =  0.256640120410225 	error =  5.311528994411674e-12 	order =  4.000017904662537
result =  0.2566401204052454 	error =  3.319566843629218e-13 	order =  4.000060312077135
result =  0.2566401204049341 	error =  2.0650148258027912e-14 	order =  4.0067709579774355
result =  0.2566401204049148 	error =  1.3322676295501878e-15 	order =  3.9541963103868754
result =  0.25664012040491346 	err